In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pydrake.all import (AddCompliantHydroelasticProperties,
                         AddContactMaterial, AddMultibodyPlant,
                         AddRigidHydroelasticProperties, Box, Capsule,
                         CollisionFilterDeclaration, ConstantValueSource,
                         ConstantVectorSource, Convex, CoulombFriction,
                         Cylinder, DeformableBodyConfig, DiagramBuilder,
                         DifferentialInverseKinematicsIntegrator,
                         DifferentialInverseKinematicsParameters,
                         DiscreteDerivative, Filament, FixedOffsetFrame,
                         GeometryInstance, GeometrySet, IllustrationProperties,
                         Integrator, LeafSystem, LinearBushingRollPitchYaw,
                         Meshcat, MeshcatVisualizer, MeshcatVisualizerParams,
                         ModelInstanceIndex, MultibodyPlant, LeafSystem, BasicVector,
                         MultibodyPlantConfig, Multiplexer, Parser,
                         PdControllerGains, PidController, PiecewisePolynomial,
                         PiecewisePose, PiecewiseQuaternionSlerp,
                         PrismaticJoint, ProximityProperties, RevoluteJoint,
                         Rgba, RigidTransform, RotationMatrix,
                         SceneGraphConfig, Simulator, SpatialInertia, Sphere,
                         Value, VectorLogSink)

In [ ]:
meshcat = Meshcat()

In [ ]:
class DesiredStateSource(LeafSystem):
    def __init__(self, speed, num_actuators):
        super().__init__()
        self._speed = speed
        self._num_actuators = num_actuators

        self.DeclareVectorOutputPort(
            "desired_state",
            BasicVector(num_actuators * 2),
            self.CalcOutput
        )

    def CalcOutput(self, context, output):
        t = context.get_time()
        value = output.get_mutable_value()

        value[:self._num_actuators] = self._speed * t
        value[self._num_actuators:] = self._speed

In [ ]:
nodes = np.loadtxt("./filament_configurations/n4.txt").T
rope = Filament(False, nodes, Filament.CircularCrossSection(diameter=3e-3))

node_pos = rope.node_pos()
edge_m1 = rope.edge_m1()
num_nodes = node_pos.shape[1]
num_edges = edge_m1.shape[1]
edge_t = []
edge_l = []
for i in range(num_edges):
    ip1 = (i + 1) % num_nodes
    l = np.linalg.norm(node_pos[:, ip1] - node_pos[:, i])
    t = (node_pos[:, ip1] - node_pos[:, i]) / l
    edge_t.append(t)
    edge_l.append(l)
edge_t = np.array(edge_t).T
edge_l = np.array(edge_l).T

In [ ]:
mass_density = 1000
diameter = 0.003
P_gain = 1e6
speed = 0.8

In [ ]:
builder = DiagramBuilder()

plant_config = MultibodyPlantConfig()
plant_config.time_step = 1e-4

plant, scene_graph = AddMultibodyPlant(plant_config, builder)
collision_filter_manager = scene_graph.collision_filter_manager()

geom_ids = []
links = []
head_frames = []
tail_frames = []
for i in range(num_edges):
    l, t, m1 = edge_l[i], edge_t[:,i], edge_m1[:,i]
    pos = (node_pos[:, i] + node_pos[:, (i+1) % num_nodes]) / 2

    name = f"link{i}"
    link = plant.AddRigidBody(name, SpatialInertia.SolidCapsuleWithDensity(mass_density, diameter/2, l, [0,0,1]))
    links.append(link)
    capsule = Capsule(diameter/2, l)
    plant.RegisterVisualGeometry(link, RigidTransform(), capsule, name, [0.6,0.6,0.4,1.0])
    geom_id = plant.RegisterCollisionGeometry(link, RigidTransform(), capsule, name, CoulombFriction(0,0))
    geom_ids.append(geom_id)
    R_WL = np.vstack([m1, np.cross(t, m1), t]).T
    X_WL = RigidTransform(RotationMatrix.MakeUnchecked(R_WL), pos)
    plant.SetDefaultFreeBodyPose(link, X_WL)
    head_frames.append(plant.AddFrame(
        FixedOffsetFrame(f"{name}_head_frame", link,RigidTransform([0, 0, l/2]))
    ))
    tail_frames.append(plant.AddFrame(
        FixedOffsetFrame(f"{name}_tail_frame", link,RigidTransform([0, 0, -l/2]))
    ))
for i in range(num_edges - 1):
    collision_filter_manager.Apply(
        CollisionFilterDeclaration().ExcludeWithin(
            GeometrySet([geom_ids[i], geom_ids[i+1]])
    ))
    plant.AddForceElement(LinearBushingRollPitchYaw(
        head_frames[i],
        tail_frames[i+1],
        torque_stiffness_constants=np.ones(3) * 2e-3,
        torque_damping_constants=np.ones(3) * 0,
        force_stiffness_constants=np.ones(3) * 3e3,
        force_damping_constants=np.ones(3) * 0,
    ))

X_WP1 = RigidTransform(
    RotationMatrix.MakeFromOneVector(node_pos[:,0] - node_pos[:,-1], 2),
    (node_pos[:,0] + node_pos[:,1]) / 2
)
joint1 = plant.AddJoint(PrismaticJoint(
    "joint1",
    plant.AddFrame(FixedOffsetFrame("world_offset_frame1", plant.world_frame(), X_WP1)),
    plant.AddFrame(FixedOffsetFrame("link1_offset_frame", links[0],
                                    plant.GetDefaultFreeBodyPose(links[0]).inverse() @ X_WP1)),
    [0,0,1]
))
actuator1_index = plant.AddJointActuator("actuator1", joint1).index()
plant.get_mutable_joint_actuator(actuator1_index).set_controller_gains(PdControllerGains(p=P_gain, d=0))

X_WP2 = RigidTransform(
    RotationMatrix.MakeFromOneVector(node_pos[:,-1] - node_pos[:,0], 2),
    (node_pos[:,-1] + node_pos[:,-2]) / 2
)
joint2 = plant.AddJoint(PrismaticJoint(
    "joint2",
    plant.AddFrame(FixedOffsetFrame("world_offset_frame2", plant.world_frame(), X_WP2)),
    plant.AddFrame(FixedOffsetFrame(f"link{len(links)-1}_offset_frame", links[-1],
                                    plant.GetDefaultFreeBodyPose(links[-1]).inverse() @ X_WP2)),
    [0,0,1]
))
actuator2_index = plant.AddJointActuator("actuator2", joint2).index()
plant.get_mutable_joint_actuator(actuator2_index).set_controller_gains(PdControllerGains(p=P_gain, d=0))

plant.mutable_gravity_field().set_gravity_vector(np.zeros(3))
plant.Finalize()

# Add position source
source = builder.AddSystem(DesiredStateSource(speed / 2, 2))
builder.Connect(source.get_output_port(), plant.get_desired_state_input_port(ModelInstanceIndex(1)))

# Add Meshcat visualizer
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, scene_graph, meshcat, MeshcatVisualizerParams(publish_period=1/500)
)

diagram = builder.Build()

simulator = Simulator(diagram)

visualizer.StartRecording()
simulator.AdvanceTo(0.7)
visualizer.StopRecording()
visualizer.PublishRecording()